In [15]:
import pymssql
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

# 数据库连接
# 数据库信息已删除。
server = ''
user = ''
password = ''
db = 'R'
connect = pymssql.connect(server, user, password, db, charset='GB18030') #解决乱码问题
if connect:
    print("连接成功！")
engine = create_engine('mssql+pymssql://' + user + ':' + password + '@' + server + '/' + db)
    
cursor = connect.cursor()   #创建一个游标对象,python里的sql语句都要通过cursor来执行

连接成功！


In [3]:
# # 读取关系数据到dataframe
# sql = "select count(1) from Relationships;"
# cursor.execute(sql)   #执行sql语句
# row = cursor.fetchone()  #读取查询结果,
# print("Relationships表的数据共%s行" % (row[0]))   #输出结果

# sql = "select * from Relationships;"
# dfRel = pd.read_sql(sql, connect)
# print("\nRelationships表的基本信息：")
# print(dfRel.info())

# print("\nRelationships表的样例数据：")
# print(dfRel.head(5))

# # 20名用户的答案表
# sql = "select * from Truth20;"
# dfTruth = pd.read_sql(sql, connect)
# print("\nTruth20表的基本信息：")
# print(dfTruth.info())

# print("\nTruth20表的样例数据：")
# print(dfTruth.head(5))

# # 20名用户的答案表
# sql = "select * from Users;"
# dfUsers = pd.read_sql(sql, connect)
# print("\nUsers表的基本信息：")
# print(dfUsers.info())

# print("\nUsers表的样例数据：")
# print(dfUsers.head(5))

# # 用于提交答案的Results表
# sql = "select * from Results;"
# dfResult = pd.read_sql(sql, connect)
# print("\nResults表的基本信息：")
# print(dfResult.info())

# print("\nResults表的样例数据：")
# print(dfResult.head(5))

# # loc函数取targetUser==1的用户的id
# targetUsers = dfUser.loc[dfUser['targetUser'] == 1, ['id']]

# # 关联二度好友
# df1 = pd.merge(targetUsers, dfRel, left_on=['id'], right_on=['userID'])
# df2 = pd.merge(df1, dfRel, left_on=['friendID'], right_on=['userID'])
# # 用group by计算共同好友数，形成分层索引；通过reset_index将series转为dataframe
# df3 = df2.groupby(['id','friendID_y'])['friendID_x'].count().reset_index()
# # 将结算结果改名为userID, ffID, cn
# df3.rename(columns={'id':'userID','friendID_y':'ffID','friendID_x':'cn'}, inplace=True)
# #print(df3.info())

# # 补充用户信息
# df4 = pd.merge(df3, dfUser, left_on='userID', right_on='id')
# df5 = pd.merge(df4, dfUser, left_on='ffID', right_on='id')

# # 关联好友表，用于标记二度好友是否为好友
# df6 = pd.merge(df5, dfRel, how='left', left_on=['userID','ffID'], right_on=['userID','friendID'])
# # 将标记改为0,1，并改名为isFriend，数据类型改为int
# df6.loc[df6['friendID'].isnull(), ['friendID']]=0
# df6.loc[df6['friendID']>0, ['friendID']]=1
# df6.rename(columns={'friendID':'isFriend'}, inplace=True)
# df6['isFriend']=df6['isFriend'].astype(int)

# # 计算指标
# df6['s_LHN1']=df6['cn']/(df6['nFriends_x']*df6['nFriends_y'])
# print(df6.loc[df6['isFriend']>0].info())a

In [4]:
# import pandas as pd
# import numpy

# # 读取本地csv文件到dataframe
# dfUser = pd.read_csv("dfUser.csv", sep='\t', encoding='utf-8')
# dfRel = pd.read_csv("dfRel.csv", sep=',', encoding='utf-8',index_col=0)
# display(dfUser.head(),dfRel.head())

In [5]:
# # 构造输出结果
# team = 'winwin'
# batch = 2
# df7 = df6.loc[df6['isFriend']==0, ['userID','ffID','s_LHN1']]
# df7['rn']=df7.groupby('userID')['s_LHN1'].rank(ascending=False, method='first')
# df7=df7.loc[df7['rn']<=10]
# df7.insert(0,'teamID',team)
# df7.insert(1,'batch',3)
# df7.rename(columns={'s_LHN1':'sIndex'},inplace=True)
# print(df7)

In [11]:
myresult=pd.read_csv("data/result.csv",index_col=0)
myresult=myresult[["userID","ffID","clf_proba"]]

def shujuzhengli(myresult):
    myresult2=pd.DataFrame([],columns=["userID","ffID","clf_proba","rn"])

    for i in list(set(myresult["userID"])):
        myresult1=myresult[myresult["userID"]==i]
        myresult1.loc[:,"rn"]=""
        for j in range(myresult1.shape[0]):
            myresult1.iloc[j,3]=j+1
        myresult2=myresult2.append(myresult1)

    myresult2.loc[:,"teamID"]="winwin"
    myresult2.loc[:,"batch"]=33    ##改batch 
    myresult2=myresult2[["teamID","batch","userID","ffID","clf_proba","rn"]]
    myresult2.columns=["teamID","batch","userID","ffID","sIndex","rn"]

    return myresult2

In [12]:
myresult2=shujuzhengli(myresult)
myresult2=myresult2[myresult2["rn"]<11]

d:\Anaconda\envs\py37\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
d:\Anaconda\envs\py37\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
# myresult2=pd.read_csv("data/try.csv",index_col=0)
# myresult2.loc[:,"teamID"]="Nothing"
# myresult2.loc[:,"batch"]=2     ##改batch 
# 向数据库上传答案
from sqlalchemy.types import VARCHAR,DECIMAL
def setdtypedict(df):
    dtypedict = {}
    for i, j in zip(df.columns, df.dtypes):
        if "object" in str(j):
            dtypedict.update({i:VARCHAR(256)})
        if "float" in str(j):
            dtypedict.update({i:DECIMAL(19, 2)})
        if "int" in str(j):
            dtypedict.update({i:DECIMAL(19)})
    return dtypedict

d_type=setdtypedict(myresult2)
myresult2.to_sql('Results', engine, index=False, if_exists='append', dtype=d_type, chunksize=1000)

In [58]:
# 进一步汇总的结果视图 scoreSheetASummary，供大家快速查看
sql = "select * from scoreSheetASummary"
print(pd.read_sql(sql, connect).sort_values(by=['pScore'],ascending=False))

             teamID  nBatch   pAT5  pAT10  pScore
0            emmemm       1  0.830  0.735   0.802
33       对不起这真的是个意外       1  0.830  0.735   0.802
31  naive_benchmark      13  0.820  0.755   0.799
7             try_1      13  0.821  0.755   0.799
13  wbc_try_is_fake       4  0.820  0.755   0.799
30              emm      12  0.821  0.735   0.795
9              一个小组       6  0.820  0.735   0.795
1           1111111       7  0.820  0.740   0.795
12              我累了       9  0.820  0.740   0.795
17              QAQ       6  0.820  0.730   0.793
39           111111       3  0.820  0.730   0.793
36            tryyy       1  0.810  0.740   0.789
26             tryy       6  0.821  0.711   0.781
24       trymybest-       4  0.821  0.690   0.781
23        trymybest       2  0.811  0.711   0.781
14            try_2       3  0.821  0.689   0.781
6            a芜湖起飞a       1  0.821  0.689   0.781
34          二营长的高射炮       9  0.790  0.700   0.763
19             temp       6  0.789  0.695   0.761


In [16]:
# 查看A排行榜 scoreSheetA是视图
# A榜只是拿出来其中20个人的答案。因此，A榜结果仅供参考。过度刷A榜是没有用的。
# 最终排行榜是B榜，是全部对象的结果。
sql = "select * from scoreSheetA"
a=pd.read_sql(sql, connect).sort_values(by=['pScore'],ascending=False)
a

,teamID,batch,pAT5,pAT10,pScore
150,对不起这真的是个意外,9,0.83,0.735,0.802
131,emm,17,0.83,0.735,0.802
351,emmemm,9,0.83,0.735,0.802
130,emm,14,0.83,0.735,0.802
313,emm,16,0.83,0.735,0.802
...,...,...,...,...,...
75,try_2,2,0.00,0.000,0.000
216,111111,3,0.00,0.000,0.000
30,崇实之光FC,6,0.00,0.000,0.000
32,0,1,0.00,0.000,0.000


In [17]:
a[a["teamID"]=="winwin"]

,teamID,batch,pAT5,pAT10,pScore
375,winwin,28,0.800,0.670,0.761
374,winwin,25,0.790,0.680,0.757
321,winwin,29,0.790,0.675,0.756
255,winwin,30,0.790,0.670,0.754
376,winwin,31,0.790,0.665,0.753
305,winwin,23,0.790,0.665,0.753
181,winwin,24,0.790,0.665,0.753
256,winwin,33,0.790,0.660,0.751
322,winwin,32,0.790,0.660,0.751
180,winwin,21,0.780,0.670,0.747


In [8]:
# 关闭数据库连接
engine.dispose
cursor.close()   
connect.close()